## Deliverable 2. Create a Customer Travel Destinations Map.

In [88]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [89]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Qaanaaq,GL,77.4840,-69.3632,0.54,91,100,3.31,overcast clouds
1,1,Abha,SA,18.2164,42.5053,59.00,67,40,16.11,light rain
2,2,Maraa,BR,-1.8333,-65.3667,77.22,94,79,1.74,broken clouds
3,3,Srednekolymsk,RU,67.4500,153.6833,25.27,72,100,13.76,overcast clouds
4,4,Yumen,CN,40.2833,97.2000,47.64,17,98,3.11,overcast clouds


In [90]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [91]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
2,2,Maraa,BR,-1.8333,-65.3667,77.22,94,79,1.74,broken clouds
7,7,Bambous Virieux,MU,-20.3428,57.7575,78.01,94,75,3.44,broken clouds
10,10,Luang Prabang,LA,19.8856,102.1347,71.82,98,100,0.58,overcast clouds
11,11,Vaini,TO,-21.2000,-175.2000,78.80,69,20,9.22,few clouds
14,14,Lorengau,PG,-2.0226,147.2712,81.86,74,83,13.00,broken clouds
15,15,Inhambane,MZ,-23.8650,35.3833,68.85,68,97,7.52,overcast clouds
16,16,Georgetown,MY,5.4112,100.3354,80.60,89,20,2.30,few clouds
18,18,Saldanha,ZA,-33.0117,17.9442,64.40,36,0,2.30,clear sky
19,19,Saint George,US,37.1041,-113.5841,66.99,18,1,8.05,clear sky
22,22,Butaritari,KI,3.0707,172.7902,81.88,76,100,5.14,light rain


In [92]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [93]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [94]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Maraa,BR,77.22,broken clouds,-1.8333,-65.3667,
7,Bambous Virieux,MU,78.01,broken clouds,-20.3428,57.7575,
10,Luang Prabang,LA,71.82,overcast clouds,19.8856,102.1347,
11,Vaini,TO,78.80,few clouds,-21.2000,-175.2000,
14,Lorengau,PG,81.86,broken clouds,-2.0226,147.2712,
15,Inhambane,MZ,68.85,overcast clouds,-23.8650,35.3833,
16,Georgetown,MY,80.60,few clouds,5.4112,100.3354,
18,Saldanha,ZA,64.40,clear sky,-33.0117,17.9442,
19,Saint George,US,66.99,clear sky,37.1041,-113.5841,
22,Butaritari,KI,81.88,light rain,3.0707,172.7902,


In [95]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping city")
        

Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city


In [96]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']


# confirm the dataframe looks like example provided in module
clean_hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
7,Bambous Virieux,MU,78.01,broken clouds,-20.3428,57.7575,Casa Tia Villa
10,Luang Prabang,LA,71.82,overcast clouds,19.8856,102.1347,Villa Oasis
11,Vaini,TO,78.80,few clouds,-21.2000,-175.2000,Keleti Beach Resort
14,Lorengau,PG,81.86,broken clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
15,Inhambane,MZ,68.85,overcast clouds,-23.8650,35.3833,Oceano
16,Georgetown,MY,80.60,few clouds,5.4112,100.3354,Cititel Penang
18,Saldanha,ZA,64.40,clear sky,-33.0117,17.9442,Blue Bay Lodge
19,Saint George,US,66.99,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
22,Butaritari,KI,81.88,light rain,3.0707,172.7902,Isles Sunset Lodge
23,Puerto Del Rosario,ES,68.00,scattered clouds,28.5004,-13.8627,Hotel JM Puerto del Rosario


In [97]:
# 8a. Create the output File (CSV)
output_data_file = "./WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [98]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [99]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))